# Análisis de Demanda Comercial No Regulada por CIIU - Industria Alimentaria

Este notebook tiene como objetivo analizar el comportamiento del consumo de energía eléctrica no regulada para los códigos CIIU relacionados con la industria alimentaria. Este análisis se enfocará en extraer, procesar y visualizar la métrica “Demanda Comercial No Regulada por CIIU” disponible a través de la API de XM, empresa administradora del mercado eléctrico colombiano.

A través de este proyecto, se busca:

1. Conectarse a la API XM y extraer los datos relacionados con los códigos CIIU correspondientes a la elaboración de productos alimenticios.
2. Filtrar y procesar los datos para obtener una frecuencia diaria, adecuada para su análisis.
3. Generar visualizaciones claras y efectivas de las series de tiempo y otras representaciones para comprender tendencias y comportamientos en el consumo energético.

Librerías utilizadas:

* pandas: Para la manipulación y análisis de datos.
* datetime: Para manejar y transformar datos temporales.
* matplotlib y seaborn: Para crear gráficos informativos y estilizados.
* pydataxm: Para interactuar y consultar datos directamente desde la API XM.

## Configuración Inicial y Dependencias

En la siguiente celda, instalaremos la librería pydataxm, diseñada para interactuar de manera sencilla con la API de XM. Esta API proporciona datos relevantes del mercado eléctrico colombiano, como información sobre generación, demanda y precios de la energía.


In [3]:
# Instalar dependencias necesarias
!pip install pydataxm
!pip install openpyxl
!pip install matplotlib seaborn pandas


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Importacion de librerías


In [5]:
import pandas as pd
import datetime as dt  
import matplotlib.pyplot as plt
import seaborn as sns
from pydataxm import * 

##  Consulta de Datos

En esta etapa, se inicializa el cliente de la API utilizando la librería pydataxm, lo que permite establecer la conexión para interactuar con los datos proporcionados por XM. Posteriormente, se realiza una consulta para obtener la colección "Demanda Comercial No Regulada por CIIU" (DemaComeNoReg), y los resultados se almacenan en un DataFrame para facilitar su análisis.


In [8]:
# Conexión al cliente API
dataxm_api = pydataxm.ReadDB() 

# Obtener la colección Demanda Comercial No Regulada por CIIU ("DemaComeNoReg") 
df = dataxm_api.get_collections('DemaComeNoReg')
df.head()

,MetricId,MetricName,Entity,MaxDays,Type,Url,Filter,MetricUnits,MetricDescription
51,DemaComeNoReg,Demanda Comercial Mercado No Regulado por Agente,Agente,31,HourlyEntities,http://servapibi.xm.com.co/hourly,Codigo Comercializador,kWh,Considera la demanda propia de cada comerciali...
53,DemaComeNoReg,Demanda Comercial Mercado No Regulado por Sistema,Sistema,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Considera la demanda propia de cada comerciali...
55,DemaComeNoReg,Demanda Comercial por CIIU,CIIU,31,HourlyEntities,http://servapibi.xm.com.co/hourly,No aplica,kWh,Considera la demanda propia de cada comerciali...
